We showed that the VGG-16 network can be used to encode images it was not trained on---a cheese grater and a foot file---as vectors that are perfectly recognized by K nearest neighbors. What do you think would happen if we used nearest neighbors (or naive Bayes or a neural network) classifier on the raw vectors?

We'll download addtional images from GitHub below

In [23]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tools.zip
!unzip tools.zip

--2022-11-07 15:45:36--  https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tools.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/tools.zip [following]
--2022-11-07 15:45:37--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/tools.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2502460 (2.4M) [application/zip]
Saving to: ‘tools.zip.3’

tools.zip.3         100%[===================>]   2.39M  --.-KB/s    in 0.06s   

2022-11-07 15:45:37 (37.8 MB/s) - ‘tools.zip.3’ saved [2502460/2502460]

Archive:  tools.zip
repl

Loading the raw image vectors as a dataset.

In [24]:
# get images
from PIL import Image
from IPython.display import display
from tensorflow import keras

# from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
         
dat = []
labs = []
imgs = []
imgflist = ["cg01", "cg02", "cg03", "cg04", "cg05", "cg06", "cg07", "cg08", "cg09", "cg10",
           "ff01", "ff02", "ff03", "ff04", "ff05", "ff06", "ff07", "ff08", "ff09", "ff10"]
for imgf in imgflist:
    img = keras.utils.load_img(imgf+".jpeg", target_size=(224,224))
    imgs.append(img)
    img_arr = np.expand_dims(keras.utils.img_to_array(img), axis=0)
#    x = preprocess_input(img_arr)
#    preds = model2.predict(x)
    dat.append(img_arr.flatten())
    labs.append(imgf[0:2] == 'ff')

Dividing the data into training and testing data.

In [25]:
train = [i for i in range(0,5)]+[i for i in range(10,15)]
test = [i for i in range(5,10)]+[i for i in range(15,20)]

X_train = [dat[inst] for inst in train]
y_train = [labs[inst] for inst in train]
X_test =  [dat[inst] for inst in test]
y_test =  [labs[inst] for inst in test]

Train using Naive Bayes.

In [26]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

0.5


Train using nearest neighbor.

In [27]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=1,metric="cosine")
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

0.4


Train using neural network.

In [28]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=[100, 100, 100, 100, 100], max_iter = 10000)
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

0.5


For comparison, here are those same algorithms using the representation that comes from VGG16.

In [29]:
!pip install keras=='2.3.1'
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import vgg16
from keras import backend as K

model = vgg16.VGG16(weights='imagenet', include_top=True)

model2 = Model(model.input, model.layers[-2].output)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Gather the vectors.

In [30]:
datraw = dat
dat = []

for img in imgs:
    img_arr = np.expand_dims(keras.utils.img_to_array(img), axis=0)
    x = preprocess_input(img_arr)
    preds = model2.predict(x)
    dat.append(preds[0])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 1s 1s/step


Make a new training/testing set.

In [31]:
train = [i for i in range(0,5)]+[i for i in range(10,15)]
test = [i for i in range(5,10)]+[i for i in range(15,20)]

X_train = [dat[inst] for inst in train]
y_train = [labs[inst] for inst in train]
X_test =  [dat[inst] for inst in test]
y_test =  [labs[inst] for inst in test]

Naive Bayes.

In [32]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

1.0


Nearest neighbors.

In [33]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=1,metric="cosine")
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

1.0


And neural networks, all perfect.

In [34]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=[100, 100, 100, 100, 100], max_iter = 10000)
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

1.0
